# US Health Medicare Payments Exploratory Data Analysis with Spark

### Examine Five Years of US Health Medicare Payments Data Using Spark


In [0]:
#Let's mount Google Drive So We can Retrieve the Data
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#There is Five Files of Health Payments Data From 2014 to 2017. Source  XXXXXXX
!ls "/content/gdrive/My Drive/Health Data/"

PGYR14_P011819.ZIP  PGYR15_P011819.ZIP	PGYR16_P011819.ZIP  PGYR17_P011819.ZIP


In [0]:
#Unzip the files
#!rm -r *
!unzip -qq "/content/gdrive/My Drive/Health Data/PGYR14_P011819.ZIP"
!unzip -qq "/content/gdrive/My Drive/Health Data/PGYR15_P011819.ZIP"
!unzip -qq "/content/gdrive/My Drive/Health Data/PGYR16_P011819.ZIP"
!unzip -qq "/content/gdrive/My Drive/Health Data/PGYR17_P011819.ZIP"

replace OP_DTL_GNRL_PGYR2014_P01182019.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
replace OP_DTL_GNRL_PGYR2015_P01182019.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
replace OP_DTL_GNRL_PGYR2016_P01182019.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
replace OP_DTL_GNRL_PGYR2017_P01182019.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [0]:
#Examine the Readme File OP_PGYR2013_README_P01182019...

!cat OP_PGYR2014_README_P01182019.txt

#Each ZIP file contains 4 CSV Data Files. We are interested in the payments file...OP_DTL_GNRL_PGYR2013_P01182019.csv

Filename: OP_PGYR2014_README_P01182019.txt
Version: 1.0
Date: January 2019

1. Program Year 2014 Data Files

This data set includes records submitted for the 2014 program year that have been matched with total confidence to a particular covered recipient (i.e., physician or teaching hospital) and displays information about that recipient. This data set includes the most recent attested-to data for Program Year 2014 as of December 31, 2018.

The data set contained in the comma-separated values (CSV) file includes only the data that is eligible for publication. Consult the Open Payments Methodology and Data Dictionary Document for an explanation of the criteria that the Centers for Medicare and Medicaid Services (CMS) used to determine what data to publish. This document can be found on the Resources page of the Open Payments website (https://www.cms.gov/OpenPayments/About/Resources.html). The Methodology and Data Dictionary Document also includes information on the data collection and r

In [0]:
!ls

gdrive
OP_DTL_GNRL_PGYR2014_P01182019.csv
OP_DTL_GNRL_PGYR2015_P01182019.csv
OP_DTL_GNRL_PGYR2016_P01182019.csv
OP_DTL_GNRL_PGYR2017_P01182019.csv
OP_DTL_OWNRSHP_PGYR2014_P01182019.csv
OP_DTL_OWNRSHP_PGYR2015_P01182019.csv
OP_DTL_OWNRSHP_PGYR2016_P01182019.csv
OP_DTL_OWNRSHP_PGYR2017_P01182019.csv
OP_DTL_RSRCH_PGYR2014_P01182019.csv
OP_DTL_RSRCH_PGYR2015_P01182019.csv
OP_DTL_RSRCH_PGYR2016_P01182019.csv
OP_DTL_RSRCH_PGYR2017_P01182019.csv
OP_PGYR2014_README_P01182019.txt
OP_PGYR2015_README_P01182019.txt
OP_PGYR2016_README_P01182019.txt
OP_PGYR2017_README_P01182019.txt
OP_REMOVED_DELETED_PGYR2014_P01182019.csv
OP_REMOVED_DELETED_PGYR2015_P01182019.csv
OP_REMOVED_DELETED_PGYR2016_P01182019.csv
OP_REMOVED_DELETED_PGYR2017_P01182019.csv
sample_data


# **Install and Load Up Spark**

In [0]:
#Install Latest Version of Spark As of Current Data. 2.4.3

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.3/spark-2.4.3-bin-hadoop2.7.tgz
!tar xf spark-2.4.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.3-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
#Import the csv files for 2014 to 2017, examine and aggregate into one file. 


df2014 = spark.read.format("csv").option("inferSchema", True).option("header", True).load('OP_DTL_GNRL_PGYR2014_P01182019.csv')
df2014.show(5)
df2015 = spark.read.format("csv").option("inferSchema", True).option("header", True).load('OP_DTL_GNRL_PGYR2015_P01182019.csv')
df2015.show(5)
df2016 = spark.read.format("csv").option("inferSchema", True).option("header", True).load('OP_DTL_GNRL_PGYR2016_P01182019.csv')
df2016.show(5)
df2017 = spark.read.format("csv").option("inferSchema", True).option("header", True).load('OP_DTL_GNRL_PGYR2017_P01182019.csv')
df2017.show(5)

+-----------+----------------------+---------------------+--------------------+----------------------+--------------------+--------------------+---------------------+-------------------+---------------------+-----------------------------------------------+-----------------------------------------------+--------------+---------------+------------------+-----------------+------------------+---------------------+----------------------+--------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+---------------------------------------------------------+-----------------------------------------------------------+-------------------------------------------------------------+--------------------------------------------------------------+----------------------------------------------------------------+---------------------------------+---------------+-----------------------------------

In [0]:
df2014.printSchema()

<bound method DataFrame.printSchema of DataFrame[Change_Type: string, Covered_Recipient_Type: string, Teaching_Hospital_CCN: int, Teaching_Hospital_ID: int, Teaching_Hospital_Name: string, Physician_Profile_ID: int, Physician_First_Name: string, Physician_Middle_Name: string, Physician_Last_Name: string, Physician_Name_Suffix: string, Recipient_Primary_Business_Street_Address_Line1: string, Recipient_Primary_Business_Street_Address_Line2: string, Recipient_City: string, Recipient_State: string, Recipient_Zip_Code: string, Recipient_Country: string, Recipient_Province: string, Recipient_Postal_Code: string, Physician_Primary_Type: string, Physician_Specialty: string, Physician_License_State_code1: string, Physician_License_State_code2: string, Physician_License_State_code3: string, Physician_License_State_code4: string, Physician_License_State_code5: string, Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name: string, Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID: bi

**Examining the tables above. It looks like new fields were added for year 2016 and 2017. Most of the fields have the same name across the files. Key fields such as Physician information and Payment information looks exactly the same across the files. We will drop fields that do not match the 2013 data and aggregate the data into one file.**

In [0]:
#Lets determine which fields are in all files and drop fields that are in not 2013.
col2014 = df2014.columns
colkeep = [column for column in df2017.columns if column in col2014]


df2014 = df2014.select(colkeep)
df2015 = df2015.select(colkeep)
df2016 = df2016.select(colkeep)
df2017 = df2017.select(colkeep)

len(df2014.columns), len(df2015.columns), len(df2016.columns), len(df2017.columns), 

(49, 49, 49, 49)

In [0]:
#total number of records
df2014.count() + df2015.count() + df2016.count() + df2017.count()

32979149

In [0]:

#Spark doesn't have a function to append multiple dataframes. So we have to use a workaround.

from functools import reduce
from pyspark.sql import DataFrame

def unionAll(*dfa):
    return reduce(DataFrame.unionAll, dfa)

df = unionAll(df2014, df2015, df2016, df2017)
df.cache()


DataFrame[Change_Type: string, Covered_Recipient_Type: string, Teaching_Hospital_CCN: int, Teaching_Hospital_ID: int, Teaching_Hospital_Name: string, Physician_Profile_ID: int, Physician_First_Name: string, Physician_Middle_Name: string, Physician_Last_Name: string, Physician_Name_Suffix: string, Recipient_Primary_Business_Street_Address_Line1: string, Recipient_Primary_Business_Street_Address_Line2: string, Recipient_City: string, Recipient_State: string, Recipient_Zip_Code: string, Recipient_Country: string, Recipient_Province: string, Recipient_Postal_Code: string, Physician_Primary_Type: string, Physician_Specialty: string, Physician_License_State_code1: string, Physician_License_State_code2: string, Physician_License_State_code3: string, Physician_License_State_code4: string, Physician_License_State_code5: string, Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name: string, Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID: string, Applicable_Manufacturer_or_Applic

In [0]:
df.count() #The number of records match the aggregate of the number of records in the individual files.

KeyboardInterrupt: ignored

In [0]:
#Lets looks at the combined dataframe
df.show(5)

KeyboardInterrupt: ignored

In [0]:
#Convert numerical fields from string to appropriate datatype
#numerical: Total_Amount_of_Payment_USDollars, Total_Amount_of_Payment_USDollars

df = df.withColumn('Total_Amount_of_Payment_USDollars', df.Total_Amount_of_Payment_USDollars.cast('Decimal'))
df = df.withColumn('Number_of_Payments_Included_in_Total_Amount', df.Number_of_Payments_Included_in_Total_Amount.cast('Decimal'))

#Change Payment data to Date Type
from pyspark.sql import functions as Func
df = df.withColumn('Date_of_Payment', Func.to_date('Date_of_Payment', 'MM/dd/yyyy'))

In [0]:
df.select('Total_Amount_of_Payment_USDollars', 'Number_of_Payments_Included_in_Total_Amount').schema()

StructType(List(StructField(Total_Amount_of_Payment_USDollars,DecimalType(10,0),true),StructField(Number_of_Payments_Included_in_Total_Amount,DecimalType(10,0),true)))

In [0]:
#Missing Values
#nonNullDF = df.fillna("--")
#display(nonNullDF)

##Exploratory Data Analysis on Nearly 4 Million Records!

In [0]:
#Look at the distribution of Total Payments (2013 to 2017)
df.describe('Total_Amount_of_Payment_USDollars').show()

In [0]:
#Total Payments By Program Year
#df.groupby('Program_year').sum('Total_Amount_of_Payment_USDollars').orderBy('Program_year', ascending = False).show()
df.groupby('Program_year').sum('Total_Amount_of_Payment_USDollars').orderBy('Program_year', ascending = False).show()

+------------+--------------------------------------+
|Program_year|sum(Total_Amount_of_Payment_USDollars)|
+------------+--------------------------------------+
|        2017|                            2814070049|
|        2016|                            2817576704|
|        2015|                            2687032307|
|        2014|                            2677298935|
|  01/18/2019|                                    18|
|        null|                                 97971|
+------------+--------------------------------------+



In [0]:
#Chart Spending by Month
ms_df = df.select(Func.date_format('Date_of_Payment','MM-yyyy').alias('Month'), 'Total_Amount_of_Payment_USDollars').groupby('Month').sum('Total_Amount_of_Payment_USDollars')
ms_df.cache()

+-------+--------------------------------------+
|  Month|sum(Total_Amount_of_Payment_USDollars)|
+-------+--------------------------------------+
|02-2017|                             217673539|
|11-2105|                                    95|
|12-2015|                             171270233|
|01-2015|                             192509866|
|01-2018|                                   145|
|05-2016|                             337787144|
|04-2015|                             227693822|
|01-2101|                                   117|
|07-2017|                             177806662|
|10-2105|                                   150|
|07-2014|                             161418995|
|   null|                                     6|
|04-2017|                             179900214|
|05-2017|                             348666665|
|09-2015|                             197821469|
|08-2017|                             326954782|
|06-2105|                                    75|
|08-2016|           

In [0]:
# Line Chart Spending By Month
import matplotlib.pyplot as plt
plt.style.use('ggplot')

#ms_df1 = ms_df.filter('Month' >= Func.lit('01/2014'))   #.filter('Month' <= Func.lit('12/2017'))
ms_df.show()

plt.plot(ms_df, Month, "sum(Total_Amount_of_Payment_USDollars")


KeyboardInterrupt: ignored

In [0]:
#What is the Total Paymnents 2014 to 2016 by Recipient State?
df.groupby('Recipient_State').sum('Total_Amount_of_Payment_USDollars').orderBy('Recipient_State', ascending = True).show()

+---------------+--------------------------------------+
|Recipient_State|sum(Total_Amount_of_Payment_USDollars)|
+---------------+--------------------------------------+
|           null|                                526486|
|       FND 216"|                                  null|
|          07728|                                  null|
|             0R|                                    30|
|          22031|                                  null|
|          98003|                                  null|
|             AA|                                  5423|
|             AE|                                 79328|
|             AK|                               3184028|
|             AL|                              86830476|
|             AP|                                 46031|
|             AR|                              46065934|
|             AS|                                   177|
|         AUSTIN|                                  null|
|             AZ|              

In [0]:
# What type of payments are made?
df.groupby('Nature_of_Payment_or_Transfer_of_Value').count().show()

In [0]:
# What is the Physician Specialty?
df.groupby('Physician_Primary_Type').count().show()

In [0]:
# What is the Physician Specialty?
df.groupby('Physician_Specialty').count().show()

46167

In [0]:
Physician_Primary_Type| Physician_Specialty